In [1]:
import torch
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm

In [2]:
# Load the pretrained model
vgg = models.vgg19(weights='DEFAULT').features

# Freeze model parameters
for param in vgg.parameters():
    param.requires_grad = False

# Transformation to preprocess images
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /Users/paul-adrienmarie/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 548M/548M [07:20<00:00, 1.30MB/s]


In [11]:
def load_image(img_path):
    image = Image.open(img_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image

# Content and style images
content_img = load_image("../data/content.jpeg")
style_img = load_image("../data/style.jpeg")

# Helper function to calculate content loss
def content_loss(target, content):
    return torch.mean((target - content) ** 2)

# Helper function to calculate style loss
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    return torch.mm(tensor, tensor.t()) / (d * h * w)

def style_loss(target, style):
    target_gram = gram_matrix(target)
    style_gram = gram_matrix(style)
    return torch.mean((target_gram - style_gram) ** 2)

# Define layers to capture
content_layers = ['conv_4']
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# Extract features from layers
def get_features(image, model):
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if f"conv_{len(features)+1}" in content_layers + style_layers:
            features[f"conv_{len(features)+1}"] = x
    return features

# Optimized input image
input_img = content_img.clone().requires_grad_(True)
optimizer = optim.LBFGS([input_img])

# Define loss weights
style_weight = 1e6
content_weight = 1

# Optimization
iterations = 300
for i in range(iterations):
    def closure():
        input_img.data.clamp_(0, 1)  # Clamp to valid range
        optimizer.zero_grad()

        # Get features
        input_features = get_features(input_img, vgg)
        content_features = get_features(content_img, vgg)
        style_features = get_features(style_img, vgg)

        # Calculate content loss
        c_loss = content_weight * content_loss(input_features['conv_4'], content_features['conv_4'])

        # Calculate style loss
        s_loss = 0
        for layer in style_layers:
            s_loss += style_loss(input_features[layer], style_features[layer])
        s_loss *= style_weight

        # Total loss
        total_loss = c_loss + s_loss
        total_loss.backward()

        return total_loss

    optimizer.step(closure)

# Convert tensor to image
output_img = input_img.clone().detach().cpu().squeeze()
output_img = transforms.ToPILImage()(output_img.clamp_(0, 1))
output_img.show()

In [5]:
import os

TypeError: dirname() missing 1 required positional argument: 'p'